In [1]:
# importeren libs
import numpy as np
import pandas as pd
import pickle
import copy

import Importeer 
import Hulpfuncties
import CrewCombined
import Dataframe
import Controleren
from Class import MogelijkeCombinaties, Rooster

In [ ]:
filehandler = open('dfSprint1_eind.obj', 'rb') 
dfSprint = pickle.load(filehandler)
filehandler = open('dfKalenderCrew1.obj', 'rb') 
dfKalenderCrew = pickle.load(filehandler)
filehandler = open('dfKalenderRoom1.obj', 'rb') 
dfKalenderRoom = pickle.load(filehandler)
filehandler = open('dfSprintAddprep1.obj', 'rb') 
dfSprintAddprep = pickle.load(filehandler)
filehandler = open('dfSprintAddpost1.obj', 'rb') 
dfSprintAddpost = pickle.load(filehandler)

In [ ]:
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name='Additional tasks')
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfAdditionalTasks = dfAdditionalTasks[dfAdditionalTasks['Task'] != 0].reset_index(drop=True)
dfAdditionalTasks['aantal keer'].replace(0, 1, inplace=True)
dfSprintAdd = pd.DataFrame((dfAdditionalTasks.loc[~dfAdditionalTasks['Task'].str.contains('S1')]) ).reset_index(drop=True)
dfSprintAdd = pd.DataFrame((dfSprintAdd[dfSprintAdd['Sprint']==1]) ).reset_index(drop=True)
dfSprintAdd['Voltooid'] = False
dfSprintAdd.loc[:, 'Crew'] = [20] * len(dfSprintAdd)
dfSprintAdd['Crew'] = dfSprintAdd['Crew'].astype(object)

In [ ]:
dfSprintAdd.iloc[:,[0,20,21,22,27,30,31]]

In [ ]:
som = pd.DataFrame(data = [range(0,13), [0]*13]).T
som.columns=['Crew', 'som']

In [ ]:
som

Dennis is de crewdirector.

In [ ]:
sum = 0
for i in range(0,10*5):
      sum += dfKalenderCrew.iloc[i,:].DagRooster.dfRooster.iloc[0,1:].sum()

In [ ]:
sum

In [ ]:
for i in range(0,1*5):
    for j in range(0,13):
        som.som.at[j] += float(dfKalenderCrew.iloc[i,:].DagRooster.dfRooster.iloc[0,j+1])
        

In [ ]:
som.som.sum()

In [ ]:
dfSprint.iloc[:,[0,20,21,22,23,25,27,28,29,30,31]]

In [ ]:
som[som.som>40]

combinaties 8 of 12 inzitten

In [ ]:
import itertools

lst = som[som.som>40].Crew.tolist()
combs = []

for i in range(1, len(lst)+1):
    els = [list(x) for x in itertools.combinations(lst, i)]
    combs.extend(els)

In [ ]:
combs

In [ ]:
a= [lijst for lijst in combs if len(lijst) == 3]
b= [lijst for lijst in a if (12 in lijst) or (8 in lijst)]

In [ ]:
def InplannenAdd(taskdf, dfKalenderCrew, dfKalenderRoom, dfAdd, crewlijst, inplannen = False): 

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempAdd = pickle.loads(pickle.dumps(dfAdd))
    gelukt = False

    dagIndex = 0
    lijstVrijeUren = [0]*16
    controle = True
    dagIndexKopie = dagIndex
    halfuur = 0
    maxDagIndex = 50
    
    while halfuur < 40*2:
        for i in crewlijst:
            if (i == 20):
                break
            print(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,crewlijst[0]+1] , dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,crewlijst[1]+1] ,dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,crewlijst[2]+1] )
            if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5)):
                print("controle wordt op false gezet, dagindex: " + str(dagIndexKopie) + " " + str(i))
                controle = False
                break 
            lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
#         print(lijstVrijeUren)
#         print("check: " + str((Hulpfuncties.contains([0], lijstVrijeUren) != False) & controle))
        if ((Hulpfuncties.contains([0], lijstVrijeUren) != False) & controle):
            rijbegin , rijeind = Hulpfuncties.contains([0], lijstVrijeUren)




            if (controle == True):
                print("volgende stap")
                for i in crewlijst:
                    if i == 20:
                        break
#                         print("halfuur: " + str(halfuur) + "crew: " + str(i) + "dagIndex: " + str(dagIndexKopie))

                    dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task 
                    dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(0.5)

#                     print(taskdf)
                    dfAdd.at[dfAdd.index[dfAdd['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                    halfuur +=1
                    if (halfuur == 40*2):
                        dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
                        print(str(taskdf.Task) + "is daadwerkelijk ingepland")

                        if inplannen:
                            return [True, dfKalenderCrew, dfKalenderRoom, dfAdd]
                        else:
                            return [True, dfTempCrew, dfTempRoom, dfTempAdd]

        else:

            dagIndexKopie+=1

            controle = True
            lijstVrijeUren = [0]*16
            if (dagIndexKopie == maxDagIndex):
                break

    print(str(taskdf.Task) + " Niet ingepland of: " + str(gelukt))
    return [gelukt, dfTempCrew, dfTempRoom, dfTempAdd]

In [ ]:
c,d,e,f = InplannenAdd(dfSprintAdd.iloc[8,:], dfKalenderCrew, dfKalenderRoom, dfSprintAdd, b[0])

In [ ]:
dfKalenderCrew.iloc[0,:].DagRooster.dfRooster.iloc[0,3+1]

In [ ]:
for i in b:
    c,d,e,f = InplannenAdd(dfSprintAdd.iloc[8,:], dfKalenderCrew, dfKalenderRoom, dfSprintAdd, i)
    if c:
        print("combinatie gevonden " + str(i))
        break

In [ ]:
dfKalenderCrew.iloc[5,:].DagRooster.dfRooster.iloc[0,0:4]

In [2]:
crew = Importeer.Crew()

In [3]:
crew

,Crew members,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6,Skill 7,Skill 8,Skill 9,...,Skill 11,Skill 12,Skill 13,mon,tue,wed,thu,fri,Additional,Uren
0,Alyssa,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,1,1,0,280
1,Chris,1,0,0,0,0,1,1,1,1,...,0,1,0,0,1,0,1,0,0,140
2,Dennis,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,0,350
3,Iris,1,0,0,0,0,1,0,1,0,...,1,0,1,1,1,0,1,0,0,210
4,Jurgen,0,0,0,0,1,0,1,1,0,...,1,0,1,0,1,1,1,0,0,210
5,Lois,1,1,1,0,0,0,1,0,1,...,0,0,0,1,1,1,1,1,must have acquired skill 8 by the end of sprint 4,350
6,Melanie,1,0,1,0,0,0,0,1,1,...,1,0,0,1,1,0,1,1,0,280
7,Mitch,1,0,1,0,0,0,0,1,1,...,1,1,1,1,1,0,1,1,0,280
8,Rudy,1,1,0,0,0,0,1,1,1,...,1,0,0,1,1,1,1,1,0,350
9,Shane,1,0,0,1,0,0,0,1,1,...,0,1,1,1,1,1,1,0,0,280
